# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096863


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:39,  3.43s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<01:05,  2.42s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:45,  1.74s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:29,  1.24s/it]

...

Rendering models:  81%|████████  | 25/31 [00:09<00:00,  6.22it/s]

Rendering models:  87%|████████▋ | 27/31 [00:09<00:00,  7.21it/s]

Rendering models:  90%|█████████ | 28/31 [00:09<00:00,  7.49it/s]

Rendering models:  97%|█████████▋| 30/31 [00:09<00:00,  9.13it/s]

not-logged-in-3b0b929f4475fd276561    0.001722
not-logged-in-3b0b929f4475fd276561    0.002675
not-logged-in-3b0b929f4475fd276561    0.003407
Jodelle                               0.031469
aliabdulhameed                        0.001268
Alexander00                           0.001072
not-logged-in-effed70e06c0f8defc75    0.002304
taysish58                             0.000503
TateK                                 0.001774
ComeMyChild                           0.001036
bryndismr                             0.002071
DrPZ                                  0.000791
not-logged-in-22877e3a7af4aafb750a    0.001780
dejansson                             0.003927
dxjdxjhhhh                            0.000635
Mckay_Oyler                           0.023820
ZannaDragon                           0.000210
not-logged-in-4607991ac840eb02289e    0.000577
homant                                0.109584
Maverna                               0.000466
not-logged-in-f8c2f88355fba4282271    0.003264
acapirala    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())